# Bibliocetas

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd

# Parametros

In [2]:
pip install unidecode

Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as pd
from unidecode import unidecode

# Lista de médicos
medicos = [
    'Luiz Guilherme de Siqueira Flora',
    'Gusttavo Fernandes',
    'Maressa Carvalho',
    'Natalia Belsuzarri',
    'Rita De Cassia Alves Bomfim',
    'Joao Jorge Lorenzini'
]

# Aplicar unidecode para remover a acentuação e converter para minúsculas
medicos = [unidecode(i).lower() for i in medicos]

# Definir especialidade e cidade
BASE_DOMAIN = "https://www.doctoralia.com.br"
SPECIALIZATION = "pisciquiatra"
CITY = "sao-jose-dos-campos"

# Gerar lista de URLs
urls = ["{}/{}/{}/{}".format(BASE_DOMAIN, '-'.join(nome.lower().split()), SPECIALIZATION, CITY) for nome in medicos]

# Criar DataFrame
list_pg = pd.DataFrame({
    'DOCTORS': medicos,
    'urls': urls
})

list_pg.head(10)

,DOCTORS,urls
0,luiz guilherme de siqueira flora,https://www.doctoralia.com.br/luiz-guilherme-d...
1,gusttavo fernandes,https://www.doctoralia.com.br/gusttavo-fernand...
2,maressa carvalho,https://www.doctoralia.com.br/maressa-carvalho...
3,natalia belsuzarri,https://www.doctoralia.com.br/natalia-belsuzar...
4,rita de cassia alves bomfim,https://www.doctoralia.com.br/rita-de-cassia-a...
5,joao jorge lorenzini,https://www.doctoralia.com.br/joao-jorge-loren...


In [4]:
# Inicializa o driver do Chrome (certifique-se de ter o ChromeDriver instalado e em seu PATH)
driver = webdriver.Chrome()

# Função para clicar em "Ver mais" um determinado número de vezes
def clicar_ver_mais(vezes):
    for _ in range(vezes):
        try:
            ver_mais_button = WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.XPATH, "//a[contains(text(),'Veja mais')]")))
            driver.execute_script("arguments[0].scrollIntoView();", ver_mais_button)
            ver_mais_button.click()
            time.sleep(2)  # Aguarda um curto intervalo para carregar mais comentários
        except:
            break

# Função para coletar os comentários de uma URL específica
def coletar_comentarios(url):
    driver.get(url)
    clicar_ver_mais(100) # Clicar em "Ver mais" 10 vezes

    comentarios = driver.find_elements(By.CLASS_NAME, 'text-break')
    lista_comentarios = []

    for comentario in comentarios:
        lista_comentarios.append(comentario.text.strip())

    return lista_comentarios

# DataFrame df1 com as colunas "medico" e "url"
# Vamos supor que este DataFrame já esteja preenchido com os dados necessários
df1 = list_pg

# Listas para armazenar os resultados
lista_medicos = []
lista_comentarios_total = []

for index, row in list_pg.iterrows():
    medico = row['DOCTORS']
    url = row['urls']

    comentarios = coletar_comentarios(url)

    lista_medicos.extend([medico] * len(comentarios))
    lista_comentarios_total.extend(comentarios)

# Fecha o navegador após coletar todos os comentários
driver.quit()

# Criar DataFrame com os resultados
df = pd.DataFrame({
    'medico': lista_medicos,
    'comentarios': lista_comentarios_total
})

In [5]:
df.head()


,medico,comentarios
0,luiz guilherme de siqueira flora,C\nCarina Marin Callegari\nPerfil verificado\n...
1,luiz guilherme de siqueira flora,A consulta com o Dr.º Luiz Guilherme se traduz...
2,luiz guilherme de siqueira flora,J\nJoão Roberto Fava Junior\nPerfil verificado...
3,luiz guilherme de siqueira flora,O melhor psiquiatra que já me atendeu em toda ...
4,luiz guilherme de siqueira flora,T\nThaisnara Magalhães Silva Moura\nPerfil ver...


In [6]:
import pandas as pd
from dateutil import parser

# Mapeamento dos nomes dos meses em português para inglês
meses_pt_para_en = {
    'janeiro': 'January',
    'fevereiro': 'February',
    'março': 'March',
    'abril': 'April',
    'maio': 'May',
    'junho': 'June',
    'julho': 'July',
    'agosto': 'August',
    'setembro': 'September',
    'outubro': 'October',
    'novembro': 'November',
    'dezembro': 'December'
}

# Função para extrair datas da string
def extrair_data(texto):
    for mes_pt, mes_en in meses_pt_para_en.items():
        texto = texto.replace(mes_pt, mes_en)
    try:
        return parser.parse(texto, fuzzy=True)
    except ValueError:
        return pd.NA

# Aplicar a função à coluna 'Texto' para extrair as datas
df['Data'] = df['comentarios'].apply(extrair_data)
df['del'] = df['Data'].isna().astype(int)
#df['Data'] = df['Data'].fillna(method='ffill')
#df = df.query("del = 1")

/opt/homebrew/Caskroom/miniforge/base/envs/R/lib/python3.12/site-packages/dateutil/parser/_parser.py:1207: UnknownTimezoneWarning: tzname E identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  warnings.warn("tzname {tzname} identified but not understood.  "


In [7]:
df.head()

,medico,comentarios,Data,del
0,luiz guilherme de siqueira flora,C\nCarina Marin Callegari\nPerfil verificado\n...,2024-03-22 00:00:00,0
1,luiz guilherme de siqueira flora,A consulta com o Dr.º Luiz Guilherme se traduz...,<NA>,1
2,luiz guilherme de siqueira flora,J\nJoão Roberto Fava Junior\nPerfil verificado...,2024-03-20 00:00:00,0
3,luiz guilherme de siqueira flora,O melhor psiquiatra que já me atendeu em toda ...,<NA>,1
4,luiz guilherme de siqueira flora,T\nThaisnara Magalhães Silva Moura\nPerfil ver...,2024-03-13 00:00:00,0


In [9]:
df['medico'].value_counts()

medico
natalia belsuzarri                  232
luiz guilherme de siqueira flora    112
rita de cassia alves bomfim         100
joao jorge lorenzini                100
gusttavo fernandes                   20
maressa carvalho                     20
Name: count, dtype: int64

In [11]:
df.to_csv("dados/comentarios_por_medico.csv", index=False)

In [13]:
filtered_df = df[df['medico'] == 'rita de cassia alves bomfim']



In [11]:
pip install wordcloud

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 169.4/169.4 kB 1.8 MB/s eta 0:00:00a 0:00:01m
Note: you may need to restart the kernel to use updated packages.


In [17]:
import nltk
from nltk.corpus import stopwords

# Download the stopwords corpus if not already downloaded
nltk.download('stopwords')

# Get the list of stopwords for Portuguese (Brazil)
stop_words = set(stopwords.words('portuguese'))

# Example text
text = "Este é um exemplo de texto em português."

# Tokenize the text
tokens = text.split()

# Remove the stopwords
filtered_tokens = [token for token in tokens if token.lower() not in stop_words]

# Join the filtered tokens back into a single string
filtered_text = ' '.join(filtered_tokens)

print(filtered_text)


ModuleNotFoundError: No module named 'nltk'